In [23]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
spark = SparkSession.builder.master("local[*]").appName("SQL 2019 Survey").getOrCreate()

In [ ]:
so_df.createTempView("so_survey_2019")
spark.sql("SELECT count(*) from so_survey_2019").show()
spark.sql("SELECT Gender, avg(Age) from so_survey_2019 group byCountry ORDER BY Respondent_Count DESC LIMIT 5").show(10)
park.sql("SELECT Country, COUNT(*) as Respondent_Count FROM so_survey_2019 GROUP BY Country ORDER BY Respondent_Count DESC LIMIT 5").show()
spark.sql("select max(Age1stCode) as oldest_age  from so_survey_2019 \
    where Country= 'United States'").show()
    spark.sql("SELECT Country, UndergradMajor, MAX(Age1stCode) as oldest_age FROM so_survey_2019 \
            WHERE ((Country = 'United States') AND (UndergradMajor IN ('Computer science, computer engineering, or software engineering', 'Mathematics or statistics')) \
            GROUP BY Country, UndergradMajor").show()
spark.sql("SELECT MAX(CAST(Age1stCode as INT)) from  so_survey_2019\
    WHERE Country = 'United States' AND (UndergradMajor = 'Computer science, computer engineering, \
        or software engineering' \
        OR UndergradMajor = 'Mathematics or statistics' group by Country").show()


In [8]:
csv_file ="/Users/sae/Downloads/stack-overflow-developer-survey-2019/survey_results_public.csv"
df = spark.read.csv(csv_file, header=True, inferSchema=True)
str(df.columns)

"['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource', 'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor', 'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode', 'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney', 'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz', 'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc', 'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan', 'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev', 'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat', 'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith', 'DatabaseDesireNextYear', 'PlatformWorkedWith', 'PlatformDesireNextYear', 'WebFrameWorkedWith', 'WebFrameDesireNextYear', 'MiscTechWorkedWith', 'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers', 'BlockchainOrg', 'BlockchainIs', 'BetterLife', 'ITperson', 'OffOn', 'SocialMedia', 'Extraversion', 'ScreenName', 'SOVisit1st', 'SOVisitFreq', 'SOVis

In [11]:
df[["Age"]].printSchema()

root
 |-- Age: string (nullable = true)



In [18]:
# fix data types: Age
df[["Age"]].groupBy("Age").count().show(100)

+----+-----+
| Age|count|
+----+-----+
|  51|  285|
|26.5|    2|
|  54|  247|
|  15|  413|
|  11|   12|
|  29| 3991|
|  69|   30|
|  42| 1024|
|  73|   18|
|  64|   68|
|   3|    3|
|  30| 4050|
|  34| 2393|
|  59|  131|
|24.5|    4|
|  28| 4387|
|  22| 3358|
|26.3|    1|
|17.5|    3|
|  35| 2388|
|  16|  588|
|  52|  326|
|  NA| 9673|
|  71|   22|
|  47|  507|
|  99|   21|
|  43|  835|
|  31| 3063|
|  18| 1160|
|  70|   28|
|  27| 4418|
|  61|   96|
|  75|   13|
|  17|  704|
|  26| 4397|
|  46|  539|
|  78|    8|
|  77|    6|
|  68|   24|
|  60|  144|
|  19| 1527|
|  23| 4109|
|  41|  976|
|23.9|    1|
|  55|  240|
|  38| 1747|
|  40| 1388|
|  25| 4669|
|  44|  699|
|  53|  282|
|  58|  171|
|  33| 2713|
|  48|  519|
|29.5|    1|
|  97|    1|
|  67|   40|
|  24| 4428|
|  32| 3055|
|   1|   18|
|  20| 2095|
|16.5|    4|
|  56|  213|
|  36| 2079|
|  10|    3|
|  37| 1807|
|46.5|    1|
|  49|  415|
|  63|   72|
|  65|   55|
|  39| 1406|
|  62|  119|
|  12|   53|
|  13|  138|
|  14|  242|

In [20]:
df = df.withColumn('Age', df.Age.cast('float'))

root
 |-- Age: float (nullable = true)



In [24]:
# fix the data types: Age1stCode
df = df.withColumn(
                "Age1stCode", when(df.Age1stCode == 'Younger than 5 years', 4)\
                .when(df.Age1stCode == "Older than 85", 86)\
                .when(df.Age1stCode == "NA", None).otherwise(df.Age1stCode)
                )

In [28]:
df = df.withColumn('Age1stCode', df.Age1stCode.cast('integer'))


In [34]:
df.createOrReplaceTempView("so_table")


In [36]:
spark.sql("SELECT Gender, AVG(Age) FROM so_table GROUP BY Gender ").show()

+--------------------+------------------+
|              Gender|          avg(Age)|
+--------------------+------------------+
|                 Man| 30.42447950143923|
|                  NA| 30.35884003741815|
|Man;Non-binary, g...|28.915243902439027|
|           Woman;Man| 26.23170731707317|
|Woman;Man;Non-bin...| 29.78723404255319|
|Woman;Non-binary,...|28.210884353741495|
|Non-binary, gende...|29.018036072144287|
|               Woman|29.495723917672105|
+--------------------+------------------+



In [44]:
spark.sql("SELECT Country, Count(Respondent) as n_respondents \
           FROM so_table \
           GROUP BY Country \
           ORDER BY n_respondents DESC \
           LIMIT 5 ").show()

+--------------+-------------+
|       Country|n_respondents|
+--------------+-------------+
| United States|        20949|
|         India|         9061|
|       Germany|         5866|
|United Kingdom|         5737|
|        Canada|         3395|
+--------------+-------------+



In [48]:
spark.sql("SELECT Country, MAX(Age1stCode) \
FROM so_table \
    WHERE Country == 'United States' AND \
    (UndergradMajor == 'Mathematics or statistics' OR \
     Undergradmajor == 'Computer science, computer engineering, or software engineering') \
    GROUP BY Country ") \
    .show()

+-------------+---------------+
|      Country|max(Age1stCode)|
+-------------+---------------+
|United States|             56|
+-------------+---------------+

